In [5]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.http import MediaFileUpload
import pandas as pd

# If modifying these scopes, delete the file token.json.
# SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

In [6]:
class gDriveApi:
    def __init__(self):
        self.SCOPES = ['https://www.googleapis.com/auth/drive']
        self.service = None
        self.parent_id = '1jHpe_YakansqGcDBlrvsSodJlb7ipHFs'
        self.folder_id = None
        self.pdf = 'application/pdf'
        self.csv = 'text/csv'
        self.child_folder_name = None
        
    def gDrive_login(self):
        creds = None
        # The file token.json stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', self.SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', self.SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.json', 'w') as token:
                token.write(creds.to_json())

        self.service = build('drive', 'v3', credentials=creds)
    
    def gDrive_readCsvFile(self,file_path):
        self.child_folder_name = pd.read_csv(file_path,header=None)[0][0]
        print(f'Child Folder Name = {self.child_folder_name}')
    
    def gDrive_createFolder(self,folder_name):
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [gDrive_obj.parent_id]
        }
        file = self.service.files().create(body=file_metadata).execute()
        print('Folder ID: %s' % file.get('id'))
        self.folder_id = file.get('id')
        print(f'Folder {folder_name} created successfully!')
        
    def gDrive_uploadfile(self,file_name,file_path,filetype,folder_id):
        file_metadata = {
            'name': file_name,
            'parents': [str(folder_id)]
        }
        
        media = MediaFileUpload(file_path , mimetype=filetype , resumable=True)
        gDrive_obj.service.files().create(
                                    body = file_metadata,
                                    media_body= media,
                                    fields = 'id').execute()
        print("file uploaded Successfully !")

In [7]:
gDrive_obj = gDriveApi()

In [8]:
gDrive_obj.gDrive_login()
gDrive_obj.gDrive_readCsvFile('E:\Git_Repositories\Freelancing\GDriveAPi\input.csv')
gDrive_obj.gDrive_createFolder(gDrive_obj.child_folder_name)
gDrive_obj.gDrive_uploadfile(file_name='demo.pdf',
                            file_path='E:\Git_Repositories\Freelancing\Elvis\Merge_Files_And_Filter_And_save_date\document_1.pdf',
                            filetype=gDrive_obj.pdf,
                            folder_id=gDrive_obj.folder_id)

Child Folder Name = First_Child
Folder ID: 10kJ0V6lQSPFpNBsjuPjlKE09-U4vc_zC
Folder First_Child created successfully!
file uploaded Successfully !
